In [33]:
import glob 
from pandas import read_csv, concat, DataFrame, __version__

#diretório
input_dir = r"C:\Users\Natan\Documents\TPAD2\dados"

##### **Listando os arquivos** #####

In [34]:
arquivos_ies = glob.glob(r"C:\Users\Natan\Documents\TPAD2\dados/*IES*.CSV")
arquivos_cursos = glob.glob(r"C:\Users\Natan\Documents\TPAD2\dados/*Cadastro_Cursos*.CSV")

##### **Filtrando colunas** #####

In [35]:
colunas_ies = ['CO_IES','SG_IES','TP_CATEGORIA_ADMINISTRATIVA','TP_ORGANIZACAO_ACADEMICA']

#Diurno
colunas_cadastro = ['NU_ANO_CENSO','SG_UF','CO_IES','QT_ING_DIURNO','QT_CONC_DIURNO','TP_ORGANIZACAO_ACADEMICA','TP_CATEGORIA_ADMINISTRATIVA','NO_CINE_ROTULO']
#Noturno
colunas_cadastro_noturno = ['NU_ANO_CENSO','SG_UF','CO_IES','QT_ING_NOTURNO','QT_CONC_NOTURNO','TP_ORGANIZACAO_ACADEMICA','TP_CATEGORIA_ADMINISTRATIVA','NO_CINE_ROTULO']

##### **Leitura dos dados e Correção de tipagem** #####

In [36]:
#Arquivos da cadastro cursos - Diurno
tabela_cadastro = []

for arquivo in arquivos_cursos:
    tabela_cadastro.append(read_csv(arquivo, sep=';', encoding='ISO 8859-1',usecols=colunas_cadastro)
    .rename(columns = {'NU_ANO_CENSO':'ano'})
    .query('TP_ORGANIZACAO_ACADEMICA==1 & TP_CATEGORIA_ADMINISTRATIVA==1')
    .groupby(['ano', 'CO_IES','NO_CINE_ROTULO']).agg(
        ingressantes_diurno = ('QT_ING_DIURNO', 'sum'),
        concluintes_diurno =('QT_CONC_DIURNO', 'sum')
        #nome_curso=('NO_CINE_ROTULO','max'))
    ))

cdd = concat(tabela_cadastro).reset_index()

#Convertendo p/ inteiros
cdd['ingressantes_diurno'] = cdd['ingressantes_diurno'].astype(int)
cdd['concluintes_diurno'] = cdd['concluintes_diurno'].astype(int)

#Arquivos da cadastro ies 
tabela_ies = []

for arquivo in arquivos_ies:
    tabela_ies.append(read_csv(arquivo, sep=';', encoding='ISO 8859-1',usecols=colunas_ies)
    .query('TP_ORGANIZACAO_ACADEMICA==1 & TP_CATEGORIA_ADMINISTRATIVA==1')
    )

ies = concat(tabela_ies)

####################################################################

#Arquivos da cadastro cursos - Noturno
tabela_cadastro_noturno = []

for arquivo2 in arquivos_cursos:
    tabela_cadastro_noturno.append(read_csv(arquivo2, sep=';', encoding='ISO 8859-1',usecols=colunas_cadastro_noturno)
    .rename(columns = {'NU_ANO_CENSO':'ano'})
    .query('TP_ORGANIZACAO_ACADEMICA==1 & TP_CATEGORIA_ADMINISTRATIVA==1')
    .groupby(['ano', 'CO_IES','NO_CINE_ROTULO']).agg(ingressantes_noturno = ('QT_ING_NOTURNO', 'sum'), concluintes_noturno =('QT_CONC_NOTURNO', 'sum'))
    )

cdn = concat(tabela_cadastro_noturno).reset_index()

#Convertendo p/ inteiros
cdn['ingressantes_noturno'] = cdn['ingressantes_noturno'].astype(int)
cdn['concluintes_noturno'] = cdn['concluintes_noturno'].astype(int)

##### **Agrupando os dados por curso e ano, nessa ordem (apenas UFPB)** #####

In [37]:
####################################################################

#Evasão - Diurno *dfd = dataframe diurno
dfd = ies.merge(cdd, on='CO_IES', how='inner')
dfd = dfd.loc[dfd['SG_IES']=='UFPB']

dfd = (dfd
.groupby(['NO_CINE_ROTULO','ano']).agg(
    ingressantes=('ingressantes_diurno','max'),
    concluintes=('concluintes_diurno','max')
).reset_index())

####################################################################

#Evasão - Noturno *dfn = dataframe noturno
dfn = ies.merge(cdn, on='CO_IES', how='inner')
dfn = dfn.loc[dfn['SG_IES']=='UFPB']

dfn = (dfn
.groupby(['NO_CINE_ROTULO','ano']).agg(
    ingressantes=('ingressantes_noturno','max'),
    concluintes=('concluintes_noturno','max')
).reset_index())

##### **Função p/ consultar cursos** #####

In [38]:
def consulta_curso():
    curso = str(input('Digite o nome do curso: '))
    turno = str(input('Digite o turno: '))
    #df = DataFrame
    #df2 = DataFrame
    
    if turno == 'diurno':
        df = DataFrame
        df = (dfd.loc[dfd['NO_CINE_ROTULO']==curso]
        .groupby(['NO_CINE_ROTULO','ano']).agg(
        ingressantes=('ingressantes','max'),
        concluintes=('concluintes','max')
        ).assign(evasão = lambda x: (1-(x.concluintes/x.ingressantes.shift(5)))*100)
        .reset_index())
        df['turno'] = 'diurno'
        return df.loc[df['ano']>2014]
        
    elif turno == 'noturno':
        df = DataFrame
        df = (dfn.loc[dfn['NO_CINE_ROTULO']==curso]
        .groupby(['NO_CINE_ROTULO','ano']).agg(
        ingressantes=('ingressantes','max'),
        concluintes=('concluintes','max')
        ).assign(evasão = lambda x: (1-(x.concluintes/x.ingressantes.shift(5)))*100)
        .reset_index())
        df['turno'] = 'noturno'
        return df.loc[df['ano']>2014]

    else:
        df = (dfd.loc[dfd['NO_CINE_ROTULO']==curso]
        .groupby(['NO_CINE_ROTULO','ano']).agg(
        ingressantes=('ingressantes','max'),
        concluintes=('concluintes','max')
        ).assign(evasão = lambda x: (1-(x.concluintes/x.ingressantes.shift(5)))*100)
        .reset_index())
        df['turno'] = 'diurno'
    
        df2 = (dfn.loc[dfn['NO_CINE_ROTULO']==curso]
        .groupby(['NO_CINE_ROTULO','ano']).agg(
        ingressantes=('ingressantes','max'),
        concluintes=('concluintes','max')
        ).assign(evasão = lambda x: (1-(x.concluintes/x.ingressantes.shift(5)))*100)
        .reset_index())
        df2['turno'] = 'noturno'
        df3 = concat([df,df2])
        return df3.dropna()

In [39]:
g = consulta_curso()
r = consulta_curso()
e = consulta_curso()
a = consulta_curso()
l = consulta_curso()
i = consulta_curso()
s = consulta_curso()
h = consulta_curso()

In [40]:
cursos = concat([g,r,e,a,l,i,s,h])
cursos.drop(index=5, inplace=True)
ccsa = cursos.filter(items=['NO_CINE_ROTULO','ano','ingressantes','evasão','turno'])
#ccsa.clip(lower=0, upper=100)
ccsa.to_excel('ccsa.xlsx')

In [1]:
pd.__version__

NameError: name 'pd' is not defined